# 双十一美妆产品数据分析

## 双十一美妆产品数据清洗

In [74]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.components import Table
from pyecharts.charts import Map, Timeline, Bar, Line, Pie, WordCloud,Tab,Page

In [3]:
df = pd.read_csv("datasets/双十一淘宝美妆数据.csv")

df.rename(columns={
    'update_time':'更新时间',
    'title':'商品',
    'price':'价格',
    'sale_count':'销售量',
    'comment_count':'评论量'
},inplace=True)

df.drop('id', axis=1, inplace=True)
df.head(5)

,更新时间,商品,价格,销售量,评论量,店名
0,2016/11/14,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2016/11/14,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2016/11/14,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2016/11/14,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2016/11/14,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂


In [4]:
print(df.dtypes)
print(f"Dataframe shape: {df.shape}")

更新时间     object
商品       object
价格      float64
销售量     float64
评论量     float64
店名       object
dtype: object
Dataframe shape: (27598, 6)


In [5]:
print(df.isnull().sum())
print(df[df.duplicated()].count())

df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df['销售量'] = df['销售量'].fillna(0)
df['评论量'] = df['评论量'].fillna(0)


更新时间       0
商品         0
价格         0
销售量     2354
评论量     2354
店名         0
dtype: int64
更新时间    101
商品      101
价格      101
销售量      82
评论量      82
店名      101
dtype: int64


In [6]:
df['更新时间'] = pd.to_datetime(df['更新时间']).apply(lambda x:x.strftime("%Y-%m-%d"))
df['销售量'] = df['销售量'].astype(int)
df['评论量'] = df['评论量'].astype(int)
df['营业额'] = df['价格'] * df['销售量']
df = df[['更新时间','商品','价格','销售量','营业额','评论量','店名']]
df.head(5)

,更新时间,商品,价格,销售量,营业额,评论量,店名
0,2016-11-14,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719,3713941.0,2704,自然堂
1,2016-11-14,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122,1575668.0,1492,自然堂
2,2016-11-14,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668,1254132.0,589,自然堂
3,2016-11-14,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805,980590.0,4287,自然堂
4,2016-11-14,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196,722244.0,618,自然堂


## 双十一美妆数据可视化EDA

### 每日美妆总销量走势

In [7]:
temp = df.groupby('更新时间').agg({'销售量':'sum'}).reset_index()
temp['销售量'] = temp['销售量'].astype(int)
temp = temp.sort_values(by='更新时间',ascending=True)

plot1 = (
    Line()
    .add_xaxis(temp['更新时间'].tolist())
    .add_yaxis("销售量", temp['销售量'].tolist())
    .set_series_opts(
        linestyle_opts=opts.LineStyleOpts(color='#5A72A0'),
        areastyle_opts=opts.AreaStyleOpts(opacity=0.6, color="#5A72A0"), #添加阴影
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_='max', name='最大销售量', itemstyle_opts=opts.ItemStyleOpts(color="#36BA98")),
                opts.MarkPointItem(type_='average', name='平均销售量', itemstyle_opts=opts.ItemStyleOpts(color='#3FA2F6')),
                opts.MarkPointItem(type_='min', name="最小销售量",itemstyle_opts=opts.ItemStyleOpts(color='#E76F51')),
            ]
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每日总销售量走势"),
        xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False)),
        yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False))
    )
)
plot1.render_notebook()

### 美妆产品累积销售量 Top10

In [8]:
datetime = df['更新时间'].unique()
datetime.sort()
datetime = datetime.tolist()
datetime

timeline = Timeline()
timeline.add_schema(is_loop_play=False, play_interval=500)
for time in datetime:
    d = df[df['更新时间'] <= time].groupby('店名').agg({'销售量':'sum', '营业额':'sum'}).sort_values(by='销售量', ascending=False).head(10).sort_values(by="销售量").to_dict()
    bar = (
        Bar()
        .add_xaxis([*d['销售量'].keys()])
        .add_yaxis("销售量", [round(val/10000,2) for val in d['销售量'].values()], 
                   label_opts=opts.LabelOpts(position="right", formatter='{@[1]/} 万个'),
                   itemstyle_opts=opts.ItemStyleOpts(color="#36BA98"))
        .add_yaxis("销售额", [round(val/10000000,2) for val in d['营业额'].values()], 
                   label_opts=opts.LabelOpts(position="right", formatter='￥{@[1]/} 亿元'),
                   itemstyle_opts=opts.ItemStyleOpts(color="#3FA2F6"))
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts(title="美妆产品累计销售量排行 TOP10"),
            xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False)),
            yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False))
        )
    )
    timeline.add(bar, time)
    
timeline.render_notebook()

In [9]:
temp3 = df.copy()
temp3 = temp3.groupby(['店名']).agg({'价格':'mean','销售量':'sum'}).sort_values('价格',ascending=True).to_dict()
temp3
barplot = (
    Bar()
        .add_xaxis([*temp3['价格'].keys()])
        .add_yaxis("价格", [round(val,2) for val in temp3['价格'].values()], 
                label_opts=opts.LabelOpts(position="right", formatter='￥{@[1]/} 元'),
                itemstyle_opts=opts.ItemStyleOpts(color="#3FA2F6"))
        .add_yaxis("销售量", [round(val/1000000,2) for val in temp3['销售量'].values()], 
                label_opts=opts.LabelOpts(position="right", formatter='{@[1]/} 万个'),
                itemstyle_opts=opts.ItemStyleOpts(color="#FF7777"))
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts(title="店家产品均价排行及产品总销量排行"),
            xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False),
                                     axislabel_opts=opts.LabelOpts(interval=50) ),
            yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False))
    )
)
barplot.render_notebook()

### 各个美妆品牌双十一期间最受欢迎的产品

In [10]:
temp4 = df.copy()
idx = temp4.groupby(['店名'])['销售量'].idxmax()
temp4 = temp4.loc[idx, ['店名', '商品', '销售量','价格']].sort_values('销售量',ascending=False)
wordcloud = (
    WordCloud()
    .add(
        "",
        [list(z) for z in zip(temp4['商品'], temp4['销售量'])],
        word_size_range=[20, 100],
        shape="diamond"
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="各美妆商家双十一最受欢迎商品"))
)

# 在 Jupyter Notebook 中渲染
wordcloud.render_notebook()

# 淘宝日用化妆数据分析

## 淘宝日用化妆数据清洗

In [11]:
df1 = pd.read_excel('datasets\淘宝日用化妆.xlsx', sheet_name='销售订单表')
df2 = pd.read_excel('datasets\淘宝日用化妆.xlsx', sheet_name="商品信息表")

### 对销售订单表进行清洗

In [12]:
df1.head()

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额
0,D31313,2019-05-16 00:00:00,S22796,东区,浙江省,台州市,X091,892,214,190888.0
1,D21329,2019-05-14 00:00:00,S11460,东区,安徽省,宿州市,X005,276,185,51060.0
2,D22372,2019-08-26 00:00:00,S11101,北区,山西省,忻州市,X078,1450,116,168200.0
3,D31078,2019-04-08 00:00:00,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,102,187068.0
4,D32470,2019-04-11 00:00:00,S18696,北区,北京市,北京市,X010,887,58,51446.0


In [14]:
print(df1.info(), "\n\n", df1.describe(),"\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31452 entries, 0 to 31451
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   订单编码    31452 non-null  object 
 1   订单日期    31452 non-null  object 
 2   客户编码    31452 non-null  object 
 3   所在区域    31450 non-null  object 
 4   所在省份    31450 non-null  object 
 5   所在地市    31452 non-null  object 
 6   商品编号    31451 non-null  object 
 7   订购数量    31450 non-null  object 
 8   订购单价    31448 non-null  object 
 9   金额      31448 non-null  float64
dtypes: float64(1), object(9)
memory usage: 2.4+ MB
None 

                   金额
count   31448.000000
mean   141653.540480
std     99627.261199
min       392.000000
25%     64963.750000
50%    120785.000000
75%    198948.250000
max    779285.000000 



In [15]:
print(f"检查是是否有重复的:\n{df1[df1.duplicated()].count()}\n")
print(f"检查是否存在missing value:\n{df1.isnull().sum()}")

#去重
df1.drop_duplicates(inplace=True)
df1.reset_index(drop=True, inplace=True)

检查是是否有重复的:
订单编码    6
订单日期    6
客户编码    6
所在区域    6
所在省份    6
所在地市    6
商品编号    6
订购数量    6
订购单价    6
金额      6
dtype: int64

检查是否存在missing value:
订单编码    0
订单日期    0
客户编码    0
所在区域    2
所在省份    2
所在地市    0
商品编号    1
订购数量    2
订购单价    4
金额      4
dtype: int64


#### 关于填补missing value的方法

1. 缺失 **所在区域** 有两个

    | | 订单编码 | 订单日期 | 客户编码 | 所在区域 | 所在省份 | 所在地市 | 订购数量 | 订购单价 | 金额 |
    | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
    | 9718 | D25844 | 2019-02-27 00:00:00	| S14821 | NaN | 江苏省	| 扬州市 | X045 | 538 | 205 | 110290.0 |
    | 9726 | D26806 | 2019-02-23 00:00:00 | S16365 | NaN | 广东省 | 湛江市 | X022 | 1542 | 158 | 243636.0 |

    经过查询，`江苏省`属于 `东区`，`广州省`属于`南区`, 根据这个信息把这两个信息给填补上, 这两行位置index 分别为 `9718` 和 `9726`


In [16]:
df1.loc[9718] = df1.loc[9718].fillna('东区')
df1.loc[9726] = df1.loc[9726].fillna('南区')

2. 缺失 **所在省份** 有两个

    |  | 订单编码 | 订单日期 | 客户编码 | 所在区域 | 所在省份 | 所在地市 | 订购数量 | 订购单价 | 金额 |
    | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
    | 9781 | D22734 | 2019-02-23 00:00:00 | S12453 | 南区 | NaN | 广州市 | X103 | 1067 | 202 | 215534.0 | 
    | 9978 | D26028 | 2019-07-13 00:00:00 | S15878 | 北区 | NaN | 北京市 | X022 | 1317 | 158 | 208086.0 |

    经过查询， index 为 `9781` 所在省份 填写为 `广东省`, index 为  `9978` 所在省份 北京市


In [17]:
df1.loc[9781] = df1.loc[9781].fillna('广东省')
df1.loc[9978] = df1.loc[9978].fillna('北京市')

3. 缺失 **商品编号**, **订购数量**， **订购单价**，和 **金额**， 我们直接去掉

In [18]:
df1.dropna(inplace=True)

In [19]:
print(f"检查是否存在missing value:\n{df1.isnull().sum()}")

检查是否存在missing value:
订单编码    0
订单日期    0
客户编码    0
所在区域    0
所在省份    0
所在地市    0
商品编号    0
订购数量    0
订购单价    0
金额      0
dtype: int64


#### 查看订单日期是否存在outlier 和 错误格式

> 发现 订单日期中 存在 `2019#3#11` 这样的格式，我们需要把它变成 `2019-3-11`
>
> 其次, 发向数据中存在一个`2050`年的数据

| | 订单编码 | 订单日期 | 客户编码 | 所在区域 | 所在省份 | 所在地市 | 商品编号 | 订购数量 | 订购单价 | 金额 | 
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 20797 | D26533 | 2050-06-09 | S21396 | 北区 | 河北省 | 石家庄市 | X022 | 759 | 158 | 119922.0 |

In [20]:
df1['订单日期'] = df1['订单日期'].apply(lambda x : pd.to_datetime(x, format='%Y#%m#%d') if isinstance(x, str) else x)
df1 = df1[df1['订单日期'] <= '2020-01-01']
print(f"订单日期的范围： {df1['订单日期'].max(), df1['订单日期'].min()}")

订单日期的范围： (Timestamp('2019-09-30 00:00:00'), Timestamp('2019-01-01 00:00:00'))


因为有些 `订购数量` 和 `订购单价` 包含了 `个` 和 `元`, 所以我们需要把它们去掉并更改它们的数据理想

In [21]:
df1['订购数量'] = df1['订购数量'].apply(lambda x: x.strip('个') if isinstance(x, str) else x).astype('int')
df1['订购单价'] = df1['订购单价'].apply(lambda x:x.strip('元') if isinstance(x, str) else x).astype('float')
df1.head(5)

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额
0,D31313,2019-05-16,S22796,东区,浙江省,台州市,X091,892,214.0,190888.0
1,D21329,2019-05-14,S11460,东区,安徽省,宿州市,X005,276,185.0,51060.0
2,D22372,2019-08-26,S11101,北区,山西省,忻州市,X078,1450,116.0,168200.0
3,D31078,2019-04-08,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,102.0,187068.0
4,D32470,2019-04-11,S18696,北区,北京市,北京市,X010,887,58.0,51446.0


### 对商品信息表进行清洗

In [22]:

print(f"检查表格是否有重复的：\n{df2[df2.duplicated()].count()}")
print(f"检查表格是否存在Missing Value:\n {df2.isnull().sum()}")

检查表格是否有重复的：
商品编号    0
商品名称    0
商品小类    0
商品大类    0
销售单价    0
dtype: int64
检查表格是否存在Missing Value:
 商品编号    0
商品名称    0
商品小类    0
商品大类    0
销售单价    0
dtype: int64


### 将两张表合并成一张

`商品编号` 作为 key

In [26]:
full_df = pd.merge(df1, df2, on='商品编号', how='left')
full_df.drop('订购单价', axis= 1, inplace=True)
full_df.head()

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,金额,商品名称,商品小类,商品大类,销售单价
0,D31313,2019-05-16,S22796,东区,浙江省,台州市,X091,892,190888.0,商品91,粉底,彩妆,214.0
1,D21329,2019-05-14,S11460,东区,安徽省,宿州市,X005,276,51060.0,商品5,面膜,护肤品,185.0
2,D22372,2019-08-26,S11101,北区,山西省,忻州市,X078,1450,168200.0,商品78,口红,彩妆,116.0
3,D31078,2019-04-08,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,187068.0,商品25,眼霜,护肤品,102.0
4,D32470,2019-04-11,S18696,北区,北京市,北京市,X010,887,51446.0,商品10,面膜,护肤品,58.0


## 淘宝日用化妆 数据可视化

In [27]:
temp = full_df.copy()
temp['订单日期'] = temp['订单日期'].apply(lambda x: x.month)
temp.head(5)

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,金额,商品名称,商品小类,商品大类,销售单价
0,D31313,5,S22796,东区,浙江省,台州市,X091,892,190888.0,商品91,粉底,彩妆,214.0
1,D21329,5,S11460,东区,安徽省,宿州市,X005,276,51060.0,商品5,面膜,护肤品,185.0
2,D22372,8,S11101,北区,山西省,忻州市,X078,1450,168200.0,商品78,口红,彩妆,116.0
3,D31078,4,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,187068.0,商品25,眼霜,护肤品,102.0
4,D32470,4,S18696,北区,北京市,北京市,X010,887,51446.0,商品10,面膜,护肤品,58.0


### 淘宝日用美妆产品每月的订购量以及销售金额

In [40]:
p1 = temp.groupby('订单日期').agg({'订购数量':'sum','金额':'sum'}).to_dict()
barplot = (
    Bar()
    .add_xaxis([f"{key} 月" for key in p1['金额'].keys()])
    .add_yaxis("订购数量 (万件)",[round(val/10000,2) for val in p1['订购数量'].values()])
    .add_yaxis("金额 (￥亿元)",[round(val/100000000,2) for val in p1['金额'].values()])
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True, position="top"))
    .set_global_opts(
        legend_opts=opts.LegendOpts(is_show=True, selected_mode="single"),
        title_opts=opts.TitleOpts(title="每月的订购量以及销售金额"),
        xaxis_opts=opts.AxisOpts(
            splitline_opts=opts.SplitLineOpts(is_show=False),
            axislabel_opts=opts.AxisLineOpts(is_show=True)),
        yaxis_opts=opts.AxisOpts(
            splitline_opts=opts.SplitLineOpts(is_show=False),
            axislabel_opts=opts.AxisLineOpts(is_show=True))
    )
    
)
barplot.render_notebook()

### 各个商品大类 & 各大类下的小类占比

In [141]:
temp2 = full_df.copy()
temp2.head(5)

data = {"彩妆":{}, "护肤品":{}}
subp2 = temp2.groupby(['商品大类','商品小类']).agg({'订购数量':'sum'}).to_dict()['订购数量']
for k, val in subp2.items():
     if k[0] in data:  # 确保大类存在于数据字典中
        data[k[0]][k[1]] = round(val, 2)



def create_main_pie(data):
    main_pie = Pie()
    main_pie.add(
        "",
        [list(z) for z in zip(data.keys(), [round(sum(cat.values())/10000,2) for cat in data.values()])]
    )
    main_pie.set_global_opts(
        title_opts=opts.TitleOpts(title="大类商品分布"),
        legend_opts=opts.LegendOpts(is_show=True)
    )
    main_pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c} 万件 {d}%"))
    return main_pie

def create_sub_pie(category, sub_data):
    sub_pie = Pie()
    sub_pie.add(
        "",
        [list(z) for z in zip(sub_data.keys(), sub_data.values())]
    )
    sub_pie.set_global_opts(
        title_opts=opts.TitleOpts(title=f"{category} 下小类商品分布"),
        legend_opts=opts.LegendOpts(is_show=True)
    )
    sub_pie.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c} 万件 {d}%"))
    return sub_pie

main_pie = create_main_pie(data)

tab = Tab()


tab.add(main_pie, "大类商品分布")

for category in data.keys():
    sub_pie = create_sub_pie(category, data[category])
    tab.add(sub_pie, f"{category} 小类商品分布")

tab.render_notebook()

### 日化产品 地区购买排行榜

In [140]:
temp3 = full_df.copy()
temp3

cities=temp3.groupby('所在地市').agg({'订购数量':'sum'}).sort_values(by='订购数量',ascending=False)[:10].sort_values(by="订购数量").to_dict()['订购数量']

city_bar=(
    Bar()
    .add_xaxis([*cities.keys()])
    .add_yaxis('订购数量',[round(val/10000,2) for val in cities.values()], label_opts=opts.LabelOpts(position="right", formatter="{@[1]/} 万"))
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts("订购数量 Top10 的市区"),
        xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False),axislabel_opts=opts.AxisLineOpts(is_show=True)),
        yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False),axislabel_opts=opts.AxisLineOpts(is_show=True))
    )
)

city_bar.render_notebook()


### 中国各个省份订购数量

In [139]:
provinces=temp3.groupby('所在省份').agg({'订购数量':'sum'}).to_dict()['订购数量']
province_map=(
    Map()
    .add('订购数量',[*provinces.items()], "china", is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="中国省份订购数量"),
        visualmap_opts=opts.VisualMapOpts(max_=1000000),
    )
)
province_map.render_notebook()


### RFM 模型 挖掘客户价值

RFM 模型是一种用于市场营销和客户分析的工具， 它用于评估客户的价值和行为。 RFM代表了以下三个指标

1. `Recency(最近购买时间)`: 客户最后一次购买时间距离当前的时间有多久，最近购买的客户被认为更有可能再次购买
2. `Frequency(购买频率)`: 用户在一定时间内购买的次数，购买频率比较高的用户通常表示他们对品牌或者产品的忠诚度比较高
3. `Monetary(购买金额)`: 用户在一定内消费的总金额，消费金额比较高的用户被认为有较高的价值的用户

In [143]:
full_df.head(5)

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,金额,商品名称,商品小类,商品大类,销售单价
0,D31313,2019-05-16,S22796,东区,浙江省,台州市,X091,892,190888.0,商品91,粉底,彩妆,214.0
1,D21329,2019-05-14,S11460,东区,安徽省,宿州市,X005,276,51060.0,商品5,面膜,护肤品,185.0
2,D22372,2019-08-26,S11101,北区,山西省,忻州市,X078,1450,168200.0,商品78,口红,彩妆,116.0
3,D31078,2019-04-08,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,187068.0,商品25,眼霜,护肤品,102.0
4,D32470,2019-04-11,S18696,北区,北京市,北京市,X010,887,51446.0,商品10,面膜,护肤品,58.0


In [148]:
rmf_df = full_df.groupby('客户编码').agg({'订单日期':'max','订单编码':'count','金额':'sum'})
rmf_df.columns=['最近购买时间','购买频率','消费金额']
rmf_df['R'] = rmf_df['最近购买时间'].rank(pct=True)
rmf_df['F'] = rmf_df['购买频率'].rank(pct=True)
rmf_df['M'] = rmf_df['消费金额'].rank(pct=True)
rmf_df['score'] = (rmf_df['R'] * 20 + rmf_df['M'] * 50 + rmf_df['F'] * 30).round(1)
rmf_df.sort_values('score', ascending=False)


,最近购买时间,购买频率,消费金额,R,F,M,score
客户编码,,,,,,,
S17476,2019-09-30,68,10258002.0,0.980166,0.985240,0.987085,98.5
S22326,2019-09-30,62,10074609.0,0.980166,0.973247,0.984317,98.0
S11581,2019-09-28,79,10333668.0,0.918358,0.996771,0.988007,97.7
S12848,2019-09-29,66,9673572.0,0.944649,0.980627,0.980627,97.3
S19095,2019-09-26,81,11031632.0,0.864852,0.999077,0.996310,97.1
...,...,...,...,...,...,...,...
S11176,2019-06-09,7,614134.0,0.035978,0.023524,0.010148,1.9
编号S13676,2019-07-12,1,334197.0,0.087177,0.000923,0.000923,1.8
S18379,2019-07-05,4,400195.0,0.071033,0.004151,0.005535,1.8
